# 如何构建用于查询分析的过滤器

我们可能希望进行查询分析以提取过滤器以传递给检索器。一种要求 LLM 表示这些过滤器的方式是使用 Pydantic 模型。然后就会出现将该 Pydantic 模型转换为可传递给检索器的过滤器的问​​题。

这可以通过手动完成，但 LangChain 还提供了一些“翻译器”，能够从通用语法翻译成特定于每个检索器的过滤器。在这里，我们将介绍如何使用这些翻译器。

In [1]:
from typing import Optional

from langchain.chains.query_constructor.ir import (
    Comparator,
    Comparison,
    Operation,
    Operator,
    StructuredQuery,
)
from langchain_community.query_constructors.chroma import ChromaTranslator
from langchain_community.query_constructors.elasticsearch import ElasticsearchTranslator
from pydantic import BaseModel

在此示例中，`year` 和 `author` 都是用于筛选的属性。

In [2]:
class Search(BaseModel):
    query: str
    start_year: Optional[int]
    author: Optional[str]

In [3]:
search_query = Search(query="RAG", start_year=2022, author="LangChain")

In [4]:
def construct_comparisons(query: Search):
    comparisons = []
    if query.start_year is not None:
        comparisons.append(
            Comparison(
                comparator=Comparator.GT,
                attribute="start_year",
                value=query.start_year,
            )
        )
    if query.author is not None:
        comparisons.append(
            Comparison(
                comparator=Comparator.EQ,
                attribute="author",
                value=query.author,
            )
        )
    return comparisons

In [5]:
comparisons = construct_comparisons(search_query)

In [6]:
_filter = Operation(operator=Operator.AND, arguments=comparisons)

In [7]:
ElasticsearchTranslator().visit_operation(_filter)

{'bool': {'must': [{'range': {'metadata.start_year': {'gt': 2022}}},
   {'term': {'metadata.author.keyword': 'LangChain'}}]}}

In [8]:
ChromaTranslator().visit_operation(_filter)

{'$and': [{'start_year': {'$gt': 2022}}, {'author': {'$eq': 'LangChain'}}]}